# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability for free (ReAct reasoning, memory, multi‑step planning).
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.

## Roadmap
1. Environment setup
2. Write simple tools and connect them to an LLM
3. Standardize tool calling by writing `to_schema`
4. Use LangChain to augment an LLM with your tools
5. Build a Perplexity‑style web‑search agent
6. (Optional) A minimal backend and frontend UI

# 1- Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Once this is done, you can select “web_agent” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, resolves the pinned dependencies, creates an isolated environment named `web_agent`, and activates it.
> * `ollama pull` downloads the model so you can run it locally without API calls.


## 1.2 Ollama setup

In this project, we start with `gemma3-1B` because it is lightweight and runs on most machines. You can try other smaller or larger LLMs such as `mistral:7b`, `phi3:mini`, or `llama3.2:1b` to compare performance. Explore available models here: https://ollama.com/library

```bash
ollama pull gemma3:1b
```

`ollama pull` downloads the model so you can run it locally without API calls.


## 2- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.


We describe each tool’s interface in the model’s prompt, defining what it does and what arguments it expects. When the model decides that a tool is needed, it emits a structured output like: `TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Francisco"}}`. Your code will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3` model how to use it when required in four steps:
1. Implement the tool
2. Create the instructions for the LLM
3. Call the LLM with the prompt
4. Parse the LLM output and call the tool

In [1]:
from openai import OpenAI

client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

In [2]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# Your goal: give the model a way to access weather information.
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Requirements:
#   • The function should be named `get_current_weather`
#   • It should take two arguments:
#         - city: str
#         - unit: str = "celsius"
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#

def get_current_weather(city: str, unit: str = "celsius") -> str:
    """Return a short, human-readable weather sentence for `city`."""
    # Deterministic stub: map unit to symbol and return a fixed message
    symbol = '°C' if unit.lower().startswith('c') else '°F'
    # Use a fixed temperature for demonstration; real API calls can replace this
    temp = 37 if symbol == '°C' else 73
    return f"It is {temp}{symbol} and sunny in {city}."


In [3]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------
# Goal:
#   Build the system and user prompts that instruct the model when and how
#   to use your tool (`get_current_weather`).
#
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describe the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# Try experimenting with different system and user prompts
# ---------------------------------------------------------

SYSTEM_PROMPT = """You are an AI assistant that helps users with weather information. 
You have access to a tool called `get_current_weather` that provides current weather details for a given
city. When you need live weather data, you MUST respond with a single line that starts with
TOOL_CALL: followed by a JSON object with the keys `name` and `args`.
Example: TOOL_CALL:{"name": "get_current_weather", "args": {"city": "San Diego"}}
Do NOT include any additional explanation when making a tool call. If you can answer without
calling the tool (general guidance), return a normal assistant response.
Use the function `get_current_weather(city: str, unit: str = "celsius")` when requesting live data.
"""

USER_QUESTION = "What is the weather in San Francisco today?"

# Messages list we will send to the LLM. The model should decide whether to call the tool.
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_QUESTION}
]

Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

Start the **Ollama** server in a terminal with `ollama serve`. This launches a local API endpoint that listens for LLM requests. Once the server is running, return to the notebook and in the next cell send a query to the model.


In [4]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Use the Ollama client to create a chat completion. 
#       - You may find some examples here: https://platform.openai.com/docs/guides/text
#       - If you are unsure, search the web for "client.chat.completions.create"
#   2. Print the raw response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------

# Send the prompt to the LLM and print the raw response
response = client.chat.completions.create(
    model="gemma3:1b",
    messages=messages,
    temperature=0
)

# Print the raw response from the model
print(response)

ChatCompletion(id='chatcmpl-63', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='TOOL_CALL:{"name": "get_current_weather", "args": {"city": "San Francisco"}}\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1761180177, model='gemma3:1b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=26, prompt_tokens=188, total_tokens=214, completion_tokens_details=None, prompt_tokens_details=None))


In [5]:
# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json


# Example: response.choices[0].message.content contains the model output
output_text = response.choices[0].message.content if hasattr(response.choices[0], 'message') else str(response)

# Search for TOOL_CALL pattern
match = re.search(r'TOOL_CALL\s*:(\{.*\})', output_text)
if match:
    tool_call_json = match.group(1)
    try:
        tool_call = json.loads(tool_call_json)
        tool_name = tool_call.get('name')
        tool_args = tool_call.get('args', {"San Fran", "celsius"})
        print(f"Calling tool `{tool_name}` with args {tool_args}")
        if tool_name == 'get_current_weather':
            result = get_current_weather(**tool_args)
            print(f"Result: {result}")
        else:
            print(f"Unknown tool: {tool_name}")
    except Exception as e:
        print(f"Error parsing TOOL_CALL JSON: {e}")
else:
    print("No TOOL_CALL found in model output.")


Calling tool `get_current_weather` with args {'city': 'San Francisco'}
Result: It is 37°C and sunny in San Francisco.


# 3- Standadize tool calling

So far, we handled tool calling manually by writing one regex and one hard-coded function. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the `TOOL_SPEC` prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function’s signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [6]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Use `inspect.signature` to get function parameters.
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing:
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect


def to_schema(fn):
    """Convert a Python function to a JSON schema for tool calling."""
    # Get function signature and docstring
    sig = inspect.signature(fn)
    doc = fn.__doc__ or ""
    
    # Build parameters schema
    params = {}
    for name, param in sig.parameters.items():
        param_type = param.annotation.__name__ if param.annotation != inspect.Parameter.empty else "string"
        params[name] = {
            "type": param_type,
            "description": f"The {name} parameter",
            "required": param.default == inspect.Parameter.empty
        }
        if param.default != inspect.Parameter.empty:
            params[name]["default"] = param.default
    
    # Build complete schema
    schema = {
        "name": fn.__name__,
        "description": doc.strip(),
        "parameters": params,
        "returns": {
            "type": sig.return_annotation.__name__ if sig.return_annotation != inspect.Parameter.empty else "string",
            "description": "The return value"
        }
    }
    return schema

# Test the schema generator with our weather function
tool_schema = to_schema(get_current_weather)
pprint(tool_schema)

{'description': 'Return a short, human-readable weather sentence for `city`.',
 'name': 'get_current_weather',
 'parameters': {'city': {'description': 'The city parameter',
                         'required': True,
                         'type': 'str'},
                'unit': {'default': 'celsius',
                         'description': 'The unit parameter',
                         'required': False,
                         'type': 'str'}},
 'returns': {'description': 'The return value', 'type': 'str'}}


In [7]:
# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user's question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the raw model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

# Get the schema for our weather tool
tool_schema = to_schema(get_current_weather)

# Create messages list with tool specification
messages_with_schema = [
    {"role": "system", "name": "tool_spec", "content": f"Available tools:\n{json.dumps(tool_schema, indent=2)}"},
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What's the weather like in Paris today?"}
]

# Call the LLM with the enhanced prompt
response = client.chat.completions.create(
    model="gemma3:1b",
    messages=messages_with_schema,
    temperature=0
)

# Print the response to see if it uses the schema correctly
print("Model Response:")
print(response.choices[0].message.content if hasattr(response.choices[0], 'message') else response)

Model Response:
TOOL_CALL:get_current_weather(city: "Paris", unit: "celsius")



## 4- LangChain for Tool Calling
So far, you built a simple tool-calling pipeline manually. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward.

In this section, you will use the **ReAct** Agent (Reasoning + Acting). It alternates between reasoning steps and tool use, producing clearer and more reliable results. We will explore reasoning-focused models in more depth next week.

The following links might be helpful:
- https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html
- https://python.langchain.com/docs/integrations/tools/
- https://python.langchain.com/docs/integrations/chat/ollama/
- https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.LLM.html

In [8]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Goal:
#   Convert your weather function into a LangChain-compatible tool.
#
# Steps:
#   1. Import `tool` from `langchain.tools`.
#   2. Keep your existing `get_current_weather` helper as before.
#   3. Create a new function (e.g., get_weather) that calls it.
#   4. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.
#   • Start with keeping the logic simple and offline-friendly.

from langchain.tools import tool

# Define a plain Python wrapper (NOT decorated) so we can register a clean
# Tool.from_function(...) with explicit parameter names. Decorating with
# @tool can create a different wrapper that leads to signature mismatches.
def get_weather(city: str, unit: str = "celsius") -> str:
    """Wrapper that forwards to get_current_weather (plain function)."""
    return get_current_weather(city, unit)


In [18]:
# ---------------------------------------------------------
# Step 2: Initialize the LangChain Agent
# ---------------------------------------------------------
# Goal:
#   Connect your tool to a local LLM using LangChain’s ReAct-style agent.
#
# Steps:
#   1. Import the required classes:
#        - ChatOllama (for local model access)
#        - initialize_agent, Tool, AgentType
#   2. Create an LLM instance (e.g., model="gemma3:1b", temperature=0).
#   3. Add your tool(s) to a list
#   4. Initialize the agent using initialize_agent
#   5. Test the agent with a natural question (e.g., "Do I need an umbrella in Seattle today?").
#
# Expected:
#   The model should reason through the question, call your tool,
#   and produce a final answer in plain language.
# ---------------------------------------------------------

from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType

# Create a ChatOllama LLM instance pointing to the local Ollama API
llm = ChatOllama(model="gemma3:1b", temperature=0)

# Wrap the Python function as a LangChain Tool using the original, typed function
# This ensures LangChain knows the parameter names and won't fall back to a
# generic `tool_input` signature that can confuse the agent.
weather_tool = Tool.from_function(get_weather, name="get_weather", description="Get current weather for a city")
print(f"Registered tool: {weather_tool.name} - {weather_tool.description}")

# Some LangChain versions use different AgentType enum members. Try the preferred name
# and fall back to a commonly available alternative to avoid AttributeError.
try:
    agent_type = AgentType.REACT_DESCRIPTION
except AttributeError:
    try:
        agent_type = AgentType.OPENAI_FUNCTIONS # ZERO_SHOT_REACT_DESCRIPTION
    except AttributeError:
        # As a last resort, use the first available AgentType enum member
        agent_type = list(AgentType)[0]

print(f"Using AgentType: {agent_type}")

agent_prompt = """
Use the following format for all tool use:

Thought: your reasoning
Action: the tool you want to use
Action Input: input to that tool
Observation: result from the tool

Repeat Thought/Action/Action Input/Observation as needed.
End with:
Thought: Final Answer.
Final Answer: your answer
"""

# Initialize the agent with the tool
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=agent_type,
    agent_prompt=agent_prompt,
    verbose=True,
    handle_parsing_errors=True
)

# Example run (commented out by default; uncomment to test)
result = agent.run("Do I need an umbrella in New York today?", )
print(result)


Registered tool: get_weather - Get current weather for a city
Using AgentType: AgentType.OPENAI_FUNCTIONS


> Entering new AgentExecutor chain...
I need a little more information to answer that for you! I don’t have real-time access to weather forecasts. 

**To tell you if you need an umbrella in New York today, I need to know:**

*   **What is the current weather forecast for New York?** (Please provide a link to a reliable weather source like Google Weather, AccuWeather, or The Weather Channel.)

Once you give me that, I can tell you if it’s likely to rain and whether you should bring an umbrella.

> Finished chain.
I need a little more information to answer that for you! I don’t have real-time access to weather forecasts. 

**To tell you if you need an umbrella in New York today, I need to know:**

*   **What is the current weather forecast for New York?** (Please provide a link to a reliable weather source like Google Weather, AccuWeather, or The Weather Channel.)

Once you give me

### What just happened?

The console log displays the **Thought → Action → Observation → …** loop until the agent produces its final answer. Because `verbose=True`, LangChain prints each intermediate reasoning step.

If you want to add more tools, simply append them to the tools list. LangChain will handle argument validation, schema generation, and tool-calling logic automatically.


## 5- Perplexity‑Style Web Search
Agents become much more powerful when they can look up real information on the web instead of relying only on their internal knowledge.

In this section, you will combine everything you have learned to build a simple Ask-the-Web Agent. You will integrate a web search tool (DuckDuckGo) and make it available to the agent using the same tool-calling approach as before.

This will let the model retrieve fresh results, reason over them, and generate an informed answer—similar to how Perplexity works.

You may find some examples from the following links:
- https://pypi.org/project/duckduckgo-search/

In [26]:
# ---------------------------------------------------------
# Step 1: Add a web search tool
# ---------------------------------------------------------
# Goal:
#   Create a tool that lets the agent search the web and return results.
#
# Steps:
#   1. Use DuckDuckGo for quick, open web searches.
#   2. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DDGS to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   3. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS as ddgs
from langchain.tools import tool
# We decorate the function with @tool so LangChain can also discover it via
# the decorator-based registration. We still unwrap the original function
# when creating a Tool.from_function to preserve the typed signature.

@tool
def search_web(query: str, max_results: int = 5) -> str:
    """Search the web (DuckDuckGo) and return a short formatted summary."""
    try:
        # Use the DDGS context manager for reliable connection handling
        with DDGS() as ddgs:
            # Get search results as a list (the _search method is more reliable)
            results = list(ddgs.text(query, max_results=max_results))
            
        if not results:
            return "No results found."

        out_lines = []
        for i, r in enumerate(results[:max_results], start=1):
            # Results dictionaries differ slightly between APIs; handle common keys
            title = r.get('title') or r.get('text') or r.get('body') or ''
            url = r.get('href') or r.get('url') or r.get('link') or ''
            snippet = r.get('body') or r.get('snippet') or r.get('text') or ''
            out_lines.append(f"{i}. {title}")
            if url:
                out_lines.append(f"   {url}")
            if snippet:
                # Keep snippet short (200 chars)
                out_lines.append(f"   {snippet[:200].strip()}")
            out_lines.append("")
        return '\n'.join(out_lines).strip()
    except Exception as e:
        # Return the error as a string so the agent can observe it instead of crashing
        return f"search_web error: {e}"

# Example usage (run in your kernel to test):
print(search_web.invoke('AI news today', max_results=3))


1. Google News - Artificial intelligence - Latest
   https://news.google.com/topics/CAAqJAgKIh5DQkFTRUFvSEwyMHZNRzFyZWhJRlpXNHRSMElvQUFQAQ
   Read full articles, watch videos, browse thousands of titles and more on the "Artificialintelligence" topic with Google News.

2. Meta AI layoffs today: 600 jobs are already being cut from ...
   https://www.fastcompany.com/91427041/meta-ai-layoffs-today-600-jobs-are-already-being-cut-from-alexandr-wang-superintelligence-lab
   1 day ago · According to a memo from Wang, the move means 'fewer conversations will be required to make a decision' as the tech giant ramps up artificial intelligence.

3. AI News & Artificial Intelligence | TechCrunch
   https://techcrunch.com/category/artificial-intelligence/
   Read the latest on artificialintelligence and machine learning tech, the companies that are building them, and the ethical issues AI raises today.

4. Meta cutting 600 AI jobs even as it continues to hire more ...
   https://apnews.com/article/me

In [ ]:

# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Goal:
#   Connect your `web_search` tool to a language model
#   so the agent can search and reason over real data.
#
# Steps:
#   1. Import `initialize_agent` and `AgentType`.
#   2. Create an LLM (e.g., ChatOllama).
#   3. Add your `web_search` tool to the tools list.
#   4. Initialize the agent using: initialize_agent
#   5. Keep `verbose=True` to observe reasoning steps.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_community.chat_models import ChatOllama

# Create local LLM
llm = ChatOllama(model="gemma3:1b", temperature=0)

# The decorator (@tool) may wrap the original function. Some LangChain helpers
# expose the original python function at attribute `func` on the decorated object;
# fall back to the function object itself if not present.
def unwrap_tool_fn(tool_obj):
    return getattr(tool_obj, 'func', tool_obj)

# Build Tool objects from the underlying functions (preserve typed signatures)
weather_tool = Tool.from_function(get_current_weather, name="get_weather", description="Get current weather for a city")
search_tool_fn = unwrap_tool_fn(search_web)
web_tool = Tool.from_function(search_tool_fn, name="search_web", description="Search the web (DuckDuckGo) and return top results")
print(f"Registered tools: {weather_tool.name}, {web_tool.name}")

# Choose AgentType robustly across LangChain versions
try:
    agent_type = AgentType.REACT_DESCRIPTION
except Exception:
    try:
        agent_type = AgentType.OPENAI_FUNCTIONS # ZERO_SHOT_REACT_DESCRIPTION
    except Exception:
        agent_type = list(AgentType)[0]
print(f"Using AgentType: {agent_type}")

# Initialize the agent with both tools
agent = initialize_agent(
    tools=[weather_tool, web_tool],
    llm=llm,
    agent=agent_type,
    verbose=True,
    handle_parsing_errors=True,
)

# Example run (uncomment to test in your environment):
# print(agent.run("Find events in NYC this week and tell me if there are any free ones."))


Registered tools: get_weather, search_web
Using AgentType: AgentType.OPENAI_FUNCTIONS


> Entering new AgentExecutor chain...
Okay, let's dive into events happening in NYC this week! Here’s a breakdown of what’s happening, with a focus on free options. Keep in mind that event listings change *quickly*, so this is based on the most current information available as of today, October 26, 2023. **Please check the official websites for the most up-to-date details before heading out!**

**Here’s a list of events, categorized by type, with a focus on free options:**

**1. Free & Community Events:**

* **Park Events:** NYC Parks regularly hosts free events in parks throughout the city. Check the NYC Parks website ([https://www.nyc.gov/site/parks/index.section?page=events](https://www.nyc.gov/site/parks/index.section?page=events)) for a calendar of events happening this week.  Look for things like:
    * **Free Concerts:** Often in Central Park or other parks.
    * **Outdoor Movie Nights:**  V


Let’s see the agent's output in action with a real example.


In [25]:
# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent
# ---------------------------------------------------------
# Goal:
#   Verify that the agent can search the web and return
#   a summarized answer based on real results.
#
# Steps:
#   1. Ask a natural question that requires live information,
#      for example: "What are the current events in San Francisco this week?"
#   2. Call agent.
#
# Expected:
#   The agent should call `web_search`, retrieve results,
#   and generate a short summary response.
# ---------------------------------------------------------

# Test the agent with a complex query that requires web search and reasoning
# query = "What are the latest AI technology announcements and tell me which ones are most significant for developers as of today?"
query = "What is the latest AI technology news as of today?" # announcements and tell me which ones are most significant for developers as of today?"
print("Asking:", query)
print("-" * 80)

# Run the agent and capture its response
response = agent.run(query)

print("\nFinal Answer:")
print("-" * 80)
print(response)

Asking: What is the latest AI technology news as of today?
--------------------------------------------------------------------------------


> Entering new AgentExecutor chain...
Okay, let's dive into some of the latest AI news as of today, October 26, 2023. It’s a really dynamic field, so here’s a breakdown of key areas and developments:

**1. ChatGPT & Large Language Models (LLMs) - Still Dominating the Conversation**

* **OpenAI's GPT-5 Rumors & Hints:** There's *still* a lot of buzz around OpenAI's next iteration of GPT. Rumors are swirling about potential improvements to reasoning, creativity, and multi-modality (handling images, audio, and video).  While there's no official announcement yet, sources suggest it could be a significant leap.
* **Microsoft's Integration:** Microsoft is aggressively integrating OpenAI's models into its products (Bing, Office, Windows).  They're focusing on making ChatGPT more accessible and integrated into existing workflows.
* **Anthropic's Claude 3


## 6- A minimal UI
This project includes a simple **React** front end that sends the user’s question to a FastAPI back end and streams the agent’s response in real time. To run the UI:

1- Open a terminal and start the Ollama server: `ollama serve`.

2- In a second terminal, navigate to the frontend folder and install dependencies:`npm install`.

3- In the same terminal, start the FastAPI back‑end: `uvicorn app:app --reload --port 8000`

4- Open a third terminal, stay in the frontend folder, and start the React dev server: `npm run dev`

5- Visit `http://localhost:5173/` in your browser.



## 🎉 Congratulations!

* You have built a **web‑enabled agent**: tool calling → JSON schema → LangChain ReAct → web search → simple UI.
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.